# Grupo 10 ~ The N-queens Problem

**SSC0713 - Sistemas Evolutivos Aplicados à Robótica**

*André Prado Furlanetti - N°USP:* 

*Mateus Doimo - N°USP: 10691971*

*Renan Peres Martins - N°USP: 10716612*

## Preparação do ambiente

Carrega as bibliotecas e inicia as constantes.

In [62]:
import numpy as np
from random import randrange, seed
from IPython.core.display import HTML

MAX_POPULATION = 50
F_PROBABILITY = 0.5
MUTATION_RATE = 1
N_SIZE = 10

# Funções do Tabuleiro

Define as funções auxiliares para manipular op tabuleiro (criação/zera, adiciona e remove peça e verifica o número de rainhas em conflito)

In [63]:
def initializeBoard():
    board = np.zeros([N_SIZE, N_SIZE], dtype=int)
    return board

def insertPiece(board, position_X, position_Y):
    if board[position_X][position_Y] != 0:
        return -1
    board[position_X][position_Y] = 1

    return board

def removePiece(board, position_X, position_Y):
    if board[position_X][position_Y] == 0:
        return -1
    board[position_X][position_Y] = 0

    return board

def verifyAttack(board, position_X, position_Y):
    atk = 0
    typeCheck = board[position_X][position_Y]

    if position_X >= N_SIZE or position_Y >= N_SIZE:
        return -1

    if typeCheck == 1:
        for i in range(N_SIZE):
            if i == position_Y:
                continue
            if board[position_X][i] != 0:
                atk += 1
        for i in range(N_SIZE):
            if i == position_X:
                continue
            if board[i][position_Y] != 0:
                atk += 1

    if typeCheck == 1:
        i = 1
        while (position_X + i < N_SIZE) and (position_Y + i < N_SIZE):
            if board[position_X + i][position_Y + i] != 0:
                atk += 1
            i += 1
        i = 1
        while (position_X - i >= 0) and (position_Y - i >= 0):
            if board[position_X - i][position_Y - i] != 0:
                atk += 1
            i += 1
        i = 1
        while (position_X + i < N_SIZE) and (position_Y - i >= 0):
            if board[position_X + i][position_Y - i] != 0:
                atk += 1
            i += 1
        i = 1
        while (position_X - i >= 0) and (position_Y + i < N_SIZE):
            if board[position_X - i][position_Y + i] != 0:
                atk += 1
            i += 1

    return atk, board



# Funções da População

Define as funções auxiliares para manipular a população de amostras (inicialização e mutação de uma população)

In [64]:
def initializePopulation():
    population = np.zeros([MAX_POPULATION, N_SIZE], dtype=int)
    for i in range(MAX_POPULATION):
        for j in range(N_SIZE):
            population[i][j] = randrange(start=0, stop=N_SIZE)
    return population

def mutation(population, mutationRate):
    for i in range(MAX_POPULATION):
        if((randrange(start=0, stop=1000)/1000.0) <= MUTATION_RATE):
            for j in range(mutationRate):
                genome = randrange(start=0, stop=N_SIZE)
                if(population[i][genome] == 0):
                    population[i][genome] += 1
                elif(population[i][genome] == N_SIZE-1):
                    population[i][genome] -= 1
                elif(randrange(start=0, stop=2)):
                    population[i][genome] += 1
                else:
                    population[i][genome] -= 1
    return population

# Funções das Amostras

In [65]:
def randomFeeding():
    vector = [randrange(start=0, stop=N_SIZE) for i in range(N_SIZE)]
    return vector

def insertSample(sample, board):
    for i in range(N_SIZE):
        board[i][sample[i]] = 1
    return board

def valuationSample(board, sample):
    sumFitness = 0
    atk = 0
    board = initializeBoard()
    for i in range(N_SIZE):
        board = insertPiece(board, i, sample[i])
    for i in range(N_SIZE):
        atk, board = verifyAttack(board, i, sample[i])
        sumFitness += atk
        board = removePiece(board, i, sample[i])
    return sumFitness, board

In [66]:
def generateFitnessVector(population, board):
    fitness_population = np.zeros(MAX_POPULATION, dtype=int)
    for i in range(MAX_POPULATION):
        fitness_population[i], board = valuationSample(board, population[i])
    return fitness_population, board
    
def recombinesIndividualsElitism(population, fitness):
    indexBetterFitness = 0
    betterFitness = 100000000

    for i in range(MAX_POPULATION):
        if(fitness[i] < betterFitness):
            indexBetterFitness = i
            betterFitness = fitness[i]
    for i in range(MAX_POPULATION):
        for j in range(N_SIZE):
            population[i][j] = population[indexBetterFitness][j]
    return indexBetterFitness, population


In [67]:
# TEM QUE TIRAR DPS
def printBoard(board):
    for i in range(N_SIZE):
        for j in range(N_SIZE):
            print(" ", board[i][j], " ", end="")
        print("")

In [68]:
#Variáveis locais
genocides = 0
stop_criterion = 1
generation = 0
mutation_rate = 1
fitness_previous = 1000000
best_individual = np.zeros(N_SIZE, dtype=int)
board = initializeBoard()

#Laço de repetição
while stop_criterion:
        population = initializePopulation()
        if generation > 0:
            if randrange(start=0, stop=2):
                for i in range(N_SIZE):
                    population[0][i] = best_individual[i]
        while True:
            fitness, board = generateFitnessVector(population, board)
            index_best_individual, population = recombinesIndividualsElitism(population,fitness)
            for i in range(N_SIZE):
                best_individual[i] = population[index_best_individual][i]
            population = mutation(population, mutation_rate)
            board = initializeBoard()
            board = insertSample(population[index_best_individual], board)
            generation += 1
            fitness_best_individual, board = valuationSample(
                board, best_individual)
            if fitness_best_individual == 0:
                board = insertSample(best_individual, board)
                printBoard(board)
                print("Generation: ", generation)
                print("genocidios: ", genocides)
                print("fitness: ", fitness_best_individual)
                stop_criterion = 0
                break
            if fitness_best_individual == fitness_previous:
                mutation_rate += 1
            elif fitness_best_individual < fitness_previous:
                mutation_rate = 0
            fitness_previous = fitness_best_individual
            if mutation_rate == N_SIZE:
                genocides += 1
                mutation_rate = 1
                population = 0
                break
print("End")

  0    0    0    0    0    1    0    0    0    0  
  0    0    0    0    0    0    0    1    0    0  
  0    0    0    0    0    0    0    0    0    1  
  0    1    0    0    0    0    0    0    0    0  
  0    0    0    1    0    0    0    0    0    0  
  0    0    0    0    0    0    0    0    1    0  
  1    0    0    0    0    0    0    0    0    0  
  0    0    1    0    0    0    0    0    0    0  
  0    0    0    0    1    0    0    0    0    0  
  0    0    0    0    0    0    1    0    0    0  
Generation:  579
genocidios:  2
fitness:  0
End


In [69]:
htmlCode = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title></title>
    <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
    <script type="text/javascript" src="./viewer/index.js"></script>
</head>
<body>
    <p>Teste</p>
    <canvas id="chess" width="800" height="800"></canvas>
    <div>
        <p>Geração: <span id="num_geracao"></span></p>
        <p>Genocidio: <span id="num_genocidio"></span></p>
        <p>Fitness: <span id="num_fitness"></span></p>
    </div>
</body>
</html> """
display(HTML(htmlCode))